KoNLPy 의 Komoran 을 이용하여 라라랜드 영화 리뷰를 토크나이징 한 텍스트를 `lalaland_komoran.txt` 에, 원문을 `lalaland.txt` 에 저장하였습니다. 총 15,595 건의 영화평입니다.

In [9]:
# Komoran tokenized La La Land review
with open('./lalaland_komoran.txt', encoding='utf-8') as f:
    sents = [sent.strip() for sent in f]

with open('./lalaland.txt', encoding='utf-8') as f:
    texts = [sent.strip() for sent in f]

print(len(sents), len(texts))

15595 15595


In [38]:
import sys
sys.path.append('C:/Users/wls40/Downloads/PyKoSpacing-master')

TextRank 에서는 word cooccurrence graph 를 만들 때 명사와 동사 등을 이용할 것을 제안하였습니다. 이는 조사나 관사와 같이 의미를 지니지 않으면서도 자주 이용되는 단어들이 높은 PageRank 를 가지게 되는 것을 방지하기 위해서입니다.

Komoran 에서의 명사 (NN), 어근 (XR), 형용사 (VA), 동사 (VV) 만을 이용하여 word cooccurrence graph 를 만듭니다. window 를 -1 로 설정하면 한 문장에서 얼마나 떨어져 있던지 상관없이 cooccurrence 를 계산하며, window 가 1 보다 클 경우에는 해당 간격만큼만 떨어진 단어들 간에만 cooccurrence 를 인정합니다.

학습 결과 `영화`, `음악`, `꿈`, `마지막` 같은 라라랜드의 엔딩을 의미하는 단어들이 핵심 단어로 선택되었습니다.

In [2]:
print('a')

a


In [11]:
from textrank import *

def komoran_tokenize(sent):
    words = sent.split()
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word, rank))

영화/NNG (1.73e+02)
보/VV (1.29e+02)
좋/VA (65.5)
하/VV (52.0)
것/NNB (47.4)
같/VA (45.4)
영화/NNP (43.8)
음악/NNG (43.6)
꿈/NNG (41.4)
있/VV (40.8)
없/VA (35.9)
마지막/NNG (31.9)
수/NNB (30.1)
사랑/NNG (28.3)
아름답/VA (26.5)
현실/NNG (24.8)
되/VV (23.9)
노래/NNG (23.4)
생각/NNG (23.2)
스토리/NNP (21.4)
번/NNB (20.3)
거/NNB (19.7)
최고/NNG (19.2)
때/NNG (19.1)
사람/NNG (19.0)
여운/NNP (17.5)
뮤지컬/NNP (16.9)
나오/VV (16.5)
듯/NNB (16.1)
영상미/NNG (16.0)


만약 모든 단어를 이용할 경우에는 조사 (JKG, JX, JKS, ... ) 어미 (EC, EP, ... ) 등이 핵심 단어로 선택됩니다. 이는 word cooccurrence graph 는 사실상 출현빈도가 높은 단어들이 높은 Rank 를 가지도록 유도하기 때문입니다.

In [12]:
keyword_extractor = KeywordSummarizer(
    tokenize=lambda x:x.split(),
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word, rank))

ㄴ/ETM (1.24e+02)
고/EC (1.03e+02)
영화/NNG (96.8)
는/ETM (94.6)
이/VCP (92.3)
이/JKS (92.0)
하/XSV (85.2)
에/JKB (79.0)
았/EP (76.1)
보/VV (73.5)
었/EP (72.8)
다/EC (68.3)
을/JKO (64.2)
하/XSA (58.8)
의/JKG (58.4)
도/JX (52.7)
ㄹ/ETM (50.2)
가/JKS (47.2)
게/EC (46.7)
는/JX (42.3)
어/EC (37.9)
좋/VA (37.6)
를/JKO (34.3)
아/EC (33.8)
은/ETM (33.7)
들/XSN (32.6)
은/JX (32.0)
하/VV (29.8)
것/NNB (26.7)
과/JC (26.5)


window 의 크기를 바꾼다 하여도 큰 변화는 없습니다. 약간의 순위 변동은 있지만, 큰 맥락이 변하지는 않습니다.

In [13]:
keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = 2,
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word, rank))

영화/NNG (1.9e+02)
보/VV (1.5e+02)
좋/VA (80.8)
하/VV (51.2)
음악/NNG (50.8)
영화/NNP (50.3)
것/NNB (44.6)
꿈/NNG (42.5)
같/VA (40.7)
있/VV (40.6)
없/VA (35.5)
마지막/NNG (33.7)
아름답/VA (32.1)
사랑/NNG (30.4)
수/NNB (29.5)
현실/NNG (27.9)
노래/NNG (26.1)
최고/NNG (23.8)
스토리/NNP (23.6)
생각/NNG (23.5)
되/VV (23.1)
번/NNB (22.7)
여운/NNP (22.1)
감동/NNG (19.1)
사람/NNG (18.6)
때/NNG (18.0)
거/NNB (18.0)
지루/XR (17.6)
영상미/NNG (16.8)
재밌/VA (16.3)


핵심 문장을 선택하기 위하여 문장 간 유사도를 계산한 다음, `min_sim` 이상의 유사도를 지니는 문장 간에 adjacent sentence graph 를 만듭니다. 그리고 여기에 PageRank 를 적용하여 핵심 문장을 선택합니다.

텍스트를 출력할 때에는 토크나이징된 문장은 가독이 어렵기 때문에 원 텍스트를 이용하여 출력합니다.

In [14]:
from textrank import KeysentenceSummarizer

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenize,
    min_sim = 0.5,
    verbose = True
)
keysents = summarizer.summarize(sents)
for idx, rank, komoran_sent in keysents:
    print('#{} ({:.3}) : {}'.format(idx, rank, texts[idx]), end='\n\n')

calculating textrank sentence similarity was done with 15595 sents
trained TextRank. n sentences = 15595
#5861 (6.12) : 사랑에 대해 다시 한 번 생각해 볼 수 있게 하는 영화인 것 같습니다 장면 처리도 좋았어요 여운이 많이 남는 영화입니다 꼭 보세요

#5947 (5.8) : 아 진짜 평점 처음 써본다 진짜 후회 안할 영화 나중에 다시 봐도 좋을것 같다 오프닝에서 신나는 노래부터 마지막의 상상 하는 씬까지 너무 좋음 결말이 여운이 있다고 해야하나 슬프다고 해야하나 꿈얘기 할때 현실성 있어서 눈물날뻔 결말이짱

#5076 (5.69) : 옛날 영화같은 느낌의 기법 사람을 행복하게 만드는 음악 약간의 촌스러움이 마음을 간지를 수 있는 지극히 현실적인 사랑과 꿈이야기를 로맨틱하게 풀어낸 영화로 연말 영화로 보기 좋은 것 같아요

#6665 (5.41) : 인생영화다 노래도 너무 좋고 배우 소품 배경 장면들 하나하나 맘에 안 드는게 없다 ㅠㅠ 특히 마지막 셉oo에서의 내용은 진짜 잊을 수가 없을 거같다 보고나면 먹먹하고 안타까운 느낌이 드는데 그래도 황홀하고 아름다운 영화다

#9271 (5.28) : 연출 음악 영상미 엔딩은 정말 좋았다 마지막에 남녀주인공이 나눈 눈빛이 아직도 잊혀지지않을만큼 여운이 남는 영화였다 초중반 약간 지루하긴했었다 배우들 춤연습을 많이한게보였음 꿈 성공 과 사랑을 다 가질수 없다는것을 현실적으로보여준 영화가아니었나싶다

#13909 (5.12) : 인생 최고의 영화 또보고싶다 영상미 음악 스토리 다 좋아요

#5922 (4.99) : 정말 영상미랑 음악은 최고였다 그리고 신선했다 음악이 너무 멋있어서 연기를 봐야 할지 노래를 들어야 할지 모를 정도로 그리고 보고 나서 생각 좀 많아진 영화 정말 이 연말에 보기 좋은 영화 인 것 같다

#11408 (4.88) : 진짜 그냥 좋았던 영화 두번봐도 재밌을영화

#12362 (4.88) : 보고 난 후

아래의 뉴스 기사에 대하여 3 개의 핵심 문장을 추출합니다.

In [6]:
sents = [
'도로에서 60대 택시기사의 얼굴을 수차례에 걸쳐 폭행해 공분을 일으킨 20대 남성이 구속됐다.', '7일 오후 서울중앙지법 서보민 영장전담 부장판사는 상해 및 공무집행방해 혐의를 받는 20대 A씨의 구속영장을 발부했다.', '서 부장판사는 “피의자가 피의사실과 같은 죄를 범하였다고 의심할 만한 상당한 이유가 있고 도망할 염려가 있어 구속의 사유와 필요성이 인정된다”고 밝혔다.', 'A씨는 앞서 지난 5일 오후 10시쯤 서울 관악구 신림동 난곡터널 인근에서 60대 택시기사 B씨를 도로에 넘어뜨리고 얼굴을 수차례 때려 상해를 입힌 혐의를 받고 있다.', '특히 A씨의 폭행 영상은 한 온라인 커뮤니티에 공유되면서 논란이 됐다. 가해자 엄벌을 촉구하는 청와대 국민청원도 올라와 하루 만에 2200명 이상의 동의를 받았다.', 'B씨는 현재 치아가 깨지고 머리 쪽을 크게 다쳐 수술이 필요할 정도의 상태인 것으로 전해졌다.', '경찰은 B씨의 상태가 호전되면 소환 조사를 진행할 방침이다.'
]

띄어쓰기 기준으로 adjacent sentence graph 를 만듭니다.

In [29]:
import kss
s = "신선 농산물과 양념 재료가 모두 들어있어 집에서 간편하게 음식을 만들어 먹을 수 있는 '밀키트' 제품이 인기를 끌고 있습니다.코로나19로 수요가 더 늘어나고 있는데요,지역 농산물로 제품을 만들다 보니 주변 농가들에도 큰 도움이 되고 있습니다. 오종우 기자의 보도입니다. '소고기 야채전골'을 해먹을 수 있는 재료와 양념이 모두 들어있는 이른바, 밀키트 제품입니다. 외식보다 저렴한 비용에 버섯과 청경채 같은 신선 농산물이 든 건강식을 먹을 수 있어 인기입니다. 밀양의 이 제조업체는 상품 출시 1년 만에 50억 원의 매출을 기록했습니다. 코로나19로 집밥 수요가 크게 늘어난 덕분입니다. [엄성민/밀양 밀키트 제조업체 : 저희가 손질이나 세척과정 없이 농산물 그대로를 사용하기 때문에 소비자분들이 원물의 신선도를 눈으로 확인하시고 섭취가 가능하십니다.] 밀양지역 농가들과 올해 맺은 계약재배 금액만 24억 원! 수확량의 절반가량은 계약재배를 한 덕분에 농민들은 가격 등락에 관계없이 농사에만 전념할 수 있습니다. [최광호/밀양 서홍감자 작목반 회장 : 1kg당 2천 원씩 계약을 해서, 일반 농가보다 한 동에, 하우스 한 동에 한 150만 원 정도 돈을 더 받았습니다. 밀키트에 들어가는 양배추와 상추, 깻잎 등 70여 가지 농산물을 밀양지역 농가에서 구매해 농민들 입장에선 든든한 거래처가 생긴 겁니다. [곽선칠/친환경농업인연합회 밀양시지부 회장 : 타지의 상인들보다도 지역 업체로서 아무래도 농민을 많이 생각하는 쪽에 서지 않나 생각합니다. 코로나19 여파로 인기를 끌고 있는 밀키트 제품이 어려움을 겪고 있는 주변 농가들에게 효자 노릇을 톡톡히 하고 있습니다. KBS 뉴스 오종우입니다."
characters = '"!?\n'

for x in range(len(characters)):
    s = s.replace(characters[x],"")

sts = []
for sent in kss.split_sentences(s):
    sts.append(sent)
print(sts)

["신선 농산물과 양념 재료가 모두 들어있어 집에서 간편하게 음식을 만들어 먹을 수 있는 '밀키트' 제품이 인기를 끌고 있습니다.", '코로나19로 수요가 더 늘어나고 있는데요,', '지역 농산물로 제품을 만들다 보니 주변 농가들에도 큰 도움이 되고 있습니다.', '오종우 기자의 보도입니다.', "'소고기 야채전골'을 해먹을 수 있는 재료와 양념이 모두 들어있는 이른바, 밀키트 제품입니다.", '외식보다 저렴한 비용에 버섯과 청경채 같은 신선 농산물이 든 건강식을 먹을 수 있어 인기입니다.', '밀양의 이 제조업체는 상품 출시 1년 만에 50억 원의 매출을 기록했습니다.', '코로나19로 집밥 수요가 크게 늘어난 덕분입니다.', '[엄성민/밀양 밀키트 제조업체 : 저희가 손질이나 세척과정 없이 농산물 그대로를 사용하기 때문에 소비자분들이 원물의 신선도를 눈으로 확인하시고 섭취가 가능하십니다.] 밀양지역 농가들과 올해 맺은 계약재배 금액만 24억 원 수확량의 절반가량은 계약재배를 한 덕분에 농민들은 가격 등락에 관계없이 농사에만 전념할 수 있습니다.', '[최광호/밀양 서홍감자 작목반 회장 : 1kg당 2천 원씩 계약을 해서, 일반 농가보다 한 동에, 하우스 한 동에 한 150만 원 정도 돈을 더 받았습니다. 밀키트에 들어가는 양배추와 상추, 깻잎 등 70여 가지 농산물을 밀양지역 농가에서 구매해 농민들 입장에선 든든한 거래처가 생긴 겁니다. [곽선칠/친환경농업인연합회 밀양시지부 회장 : 타지의 상인들보다도 지역 업체로서 아무래도 농민을 많이 생각하는 쪽에 서지 않나 생각합니다.', '코로나19 여파로 인기를 끌고 있는 밀키트 제품이 어려움을 겪고 있는 주변 농가들에게 효자 노릇을 톡톡히 하고 있습니다.', 'KBS 뉴스 오종우입니다.']


In [20]:

sents = "신선 농산물과 양념 재료가 모두 들어있어 집에서 간편하게 음식을 만들어 먹을 수 있는 '밀키트' 제품이 인기를 끌고 있습니다.코로나19로 수요가 더 늘어나고 있는데요,지역 농산물로 제품을 만들다 보니 주변 농가들에도 큰 도움이 되고 있습니다. 오종우 기자의 보도입니다. '소고기 야채전골'을 해먹을 수 있는 재료와 양념이 모두 들어있는 이른바, 밀키트 제품입니다. 외식보다 저렴한 비용에 버섯과 청경채 같은 신선 농산물이 든 건강식을 먹을 수 있어 인기입니다. 밀양의 이 제조업체는 상품 출시 1년 만에 50억 원의 매출을 기록했습니다. 코로나19로 집밥 수요가 크게 늘어난 덕분입니다. 엄성민/밀양 밀키트 제조업체 : 저희가 손질이나 세척과정 없이 농산물 그대로를 사용하기 때문에 소비자분들이 원물의 신선도를 눈으로 확인하시고 섭취가 가능하십니다. 밀양지역 농가들과 올해 맺은 계약재배 금액만 24억 원! 수확량의 절반가량은 계약재배를 한 덕분에 농민들은 가격 등락에 관계없이 농사에만 전념할 수 있습니다. 최광호/밀양 서홍감자 작목반 회장 : 1kg당 2천 원씩 계약을 해서, 일반 농가보다 한 동에, 하우스 한 동에 한 150만 원 정도 돈을 더 받았습니다. 밀키트에 들어가는 양배추와 상추, 깻잎 등 70여 가지 농산물을 밀양지역 농가에서 구매해 농민들 입장에선 든든한 거래처가 생긴 겁니다. 곽선칠/친환경농업인연합회 밀양시지부 회장 : 타지의 상인들보다도 지역 업체로서 아무래도 농민을 많이 생각하는 쪽에 서지 않나 생각합니다. 코로나19 여파로 인기를 끌고 있는 밀키트 제품이 어려움을 겪고 있는 주변 농가들에게 효자 노릇을 톡톡히 하고 있습니다. KBS 뉴스 오종우입니다."

sts = []
for sent in kss.split_sentences(sents):
    sts.append(sent)

summarizer = KeysentenceSummarizer(
    tokenize = lambda x:x.split(),
    min_sim = 0.3,
    verbose = False
)
keysents = summarizer.summarize(sts, topk=3)
for _, _, sent in keysents:
    print(sent)

신선 농산물과 양념 재료가 모두 들어있어 집에서 간편하게 음식을 만들어 먹을 수 있는 '밀키트' 제품이 인기를 끌고 있습니다.
[엄성민/밀양 밀키트 제조업체 : 저희가 손질이나 세척과정 없이 농산물 그대로를 사용하기 때문에 소비자분들이 원물의 신선도를 눈으로 확인하시고 섭취가 가능하십니다.] 밀양지역 농가들과 올해 맺은 계약재배 금액만 24억 원! 수확량의 절반가량은 계약재배를 한 덕분에 농민들은 가격 등락에 관계없이 농사에만 전념할 수 있습니다.
코로나19 여파로 인기를 끌고 있는 밀키트 제품이 어려움을 겪고 있는 주변 농가들에게 효자 노릇을 톡톡히 하고 있습니다.


In [25]:
sents

['도로에서 60대 택시기사의 얼굴을 수차례에 걸쳐 폭행해 공분을 일으킨 20대 남성이 구속됐다.',
 '7일 오후 서울중앙지법 서보민 영장전담 부장판사는 상해 및 공무집행방해 혐의를 받는 20대 A씨의 구속영장을 발부했다.',
 '서 부장판사는 “피의자가 피의사실과 같은 죄를 범하였다고 의심할 만한 상당한 이유가 있고 도망할 염려가 있어 구속의 사유와 필요성이 인정된다”고 밝혔다.',
 'A씨는 앞서 지난 5일 오후 10시쯤 서울 관악구 신림동 난곡터널 인근에서 60대 택시기사 B씨를 도로에 넘어뜨리고 얼굴을 수차례 때려 상해를 입힌 혐의를 받고 있다.',
 '특히 A씨의 폭행 영상은 한 온라인 커뮤니티에 공유되면서 논란이 됐다. 가해자 엄벌을 촉구하는 청와대 국민청원도 올라와 하루 만에 2200명 이상의 동의를 받았다.',
 'B씨는 현재 치아가 깨지고 머리 쪽을 크게 다쳐 수술이 필요할 정도의 상태인 것으로 전해졌다.',
 '경찰은 B씨의 상태가 호전되면 소환 조사를 진행할 방침이다.']

In [39]:
from pykospacing import Spacing

NotImplementedError: Cannot convert a symbolic Tensor (gru_1/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

kospacing_sent = Spacing(new_sent)
print(sent)
print(kospacing_sent)

In [15]:
from textrank import KeysentenceSummarizer

summarizer = KeysentenceSummarizer(
    tokenize = lambda x:x.split(),
    min_sim = 0.3,
    verbose = False
)
keysents = summarizer.summarize(sts, topk=3)
for _, _, sent in keysents:
    print(sent)

신선 농산물과 양념 재료가 모두 들어있어 집에서 간편하게 음식을 만들어 먹을 수 있는 '밀키트' 제품이 인기를 끌고 있습니다.
[엄성민/밀양 밀키트 제조업체 : 저희가 손질이나 세척과정 없이 농산물 그대로를 사용하기 때문에 소비자분들이 원물의 신선도를 눈으로 확인하시고 섭취가 가능하십니다.] 밀양지역 농가들과 올해 맺은 계약재배 금액만 24억 원! 수확량의 절반가량은 계약재배를 한 덕분에 농민들은 가격 등락에 관계없이 농사에만 전념할 수 있습니다.
코로나19 여파로 인기를 끌고 있는 밀키트 제품이 어려움을 겪고 있는 주변 농가들에게 효자 노릇을 톡톡히 하고 있습니다.


In [11]:
sents[1]

'7일 오후 서울중앙지법 서보민 영장전담 부장판사는 상해 및 공무집행방해 혐의를 받는 20대 A씨의 구속영장을 발부했다.'

KoNLPy 의 Komoran 을 이용하여 토크나이징과 핵심문장을 한 번에 추출하는 예시입니다.

In [8]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)
keysents = summarizer.summarize(sents, topk=3)
for _, _, sent in keysents:
    print(sent)

ImportError: DLL load failed while importing _jpype: DLL 초기화 루틴을 실행할 수 없습니다.

사실 위의 결과를 얻기 위해서는 토크나이저도 제대로 구축할 필요가 없습니다. 어자피 많이 등장한 단어라면 해당 단어를 구성하는 부분어절 (subword) 역시 자주 등장하였을 것이며, 이를 이용한 문장 간 유사도를 측정하여도 비슷하기 때문입니다.

아래는 띄어쓰기 기준으로 나뉘어진 어절에서 3음절의 subwords 를 잘라내는 토크나이저 입니다.

In [4]:
def subword_tokenizer(sent, n=3):
    def subword(token, n):
        if len(token) <= n:
            return [token]
        return [token[i:i+n] for i in range(len(token) - n + 1)]
    return [sub for token in sent.split() for sub in subword(token, n)]

subword_tokenizer('이것은 부분단어의 예시입니다 짧은 어절은 그대로 나옵니다')

['이것은',
 '부분단',
 '분단어',
 '단어의',
 '예시입',
 '시입니',
 '입니다',
 '짧은',
 '어절은',
 '그대로',
 '나옵니',
 '옵니다']

이를 이용하여도 핵심 문장은 위와 동일하게 출력됩니다.

In [5]:
summarizer = KeysentenceSummarizer(
    tokenize = subword_tokenizer,
    min_sim = 0.3,
    verbose = False
)
keysents = summarizer.summarize(sents, topk=3)
for _, _, sent in keysents:
    print(sent)

오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다
용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기
총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다


summarizer 의 R 에는 각 문장 별 중요도 (PageRank 값) 가 저장되어 있습니다.

In [6]:
summarizer.R

array([1.76438621, 0.74969733, 1.33782296, 0.61722741, 0.7377122 ,
       1.07534516, 0.62928904, 1.71145208, 1.07601036, 1.13590053,
       0.94446938, 0.67686714, 0.7008805 , 1.02103025, 1.61461996,
       0.76911158, 0.78024047, 0.65793743, 1.02927478, 0.97072522])

문장의 위치에 따라 중요도를 다르게 설정할 수도 있습니다. 뉴스 기사는 대부분 첫 문장이 중요합니다. 실제로 위의 예시에서도 첫 문장이 가장 중요한 핵심 문장으로 선택되었습니다. 만약 마지막 문장이 중요하다고 가정한다면 이러한 정보를 bias 에 추가할 수 있습니다. numpy.ndarray 형태로 bias 를 만듭니다. 마지막 문장이 다른 문장보다 10 배 중요하다고 가정하였습니다. 이를 summarize 함수의 bias 에 입력하면 가장 먼저 맨 마지막 문장이 중요한 문장으로 선택됩니다. 다른 문장들 중에서도 맨 마지막 문장과 비슷할수록 상대적인 중요도가 더 커집니다.

In [7]:
import numpy as np

bias = np.ones(len(sents))
bias[-1] = 10

keysents = summarizer.summarize(sents, topk=3, bias=bias)
for _, _, sent in keysents:
    print(sent)

일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다
경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다


R 을 다시 확인해보면 PageRank 값이 달라졌음을 확인할 수 있습니다. 상대적인 위치 외에도 특정 단어가 포함된 문장에 preference (bias) 를 더 높게 설정할 수도 있습니다.

In [8]:
summarizer.R

array([1.22183954, 0.51902092, 0.92584783, 0.42671701, 0.50982682,
       0.74430683, 0.43498201, 1.18547126, 0.74505343, 0.78632222,
       0.65347844, 0.46802437, 0.48465947, 0.70684359, 1.11845189,
       0.53125081, 0.53956034, 0.45476333, 3.14941282, 4.39416707])